In [1]:
import uproot
import numpy as np
import awkward as ak
import math as m
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep as hep
import os
import numba as nb
import boost_histogram as bh
import yaml
import json
from coffea import hist as chist
from coffea.nanoevents.methods import vector
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, TreeMakerSchema

import sys
import time

from coffea.util import save
from coffea.util import load

from yty_hist_class import yty_histplot

In [2]:
pi = 3.14159265

def Delta_phi(phi1,phi2):
    a = abs(phi1-phi2)
    return np.array(1*(a<=pi)*a+1*(a>pi)*(2*pi-a))

def Delta_eta(eta1,eta2):
    return np.array(abs(eta1-eta2))

def Delta_r(eta1,phi1,eta2,phi2):
    de = Delta_eta(eta1,eta2)
    dp = Delta_phi(phi1,phi2)
    return np.sqrt(de**2+dp**2)

In [3]:
def mkdir(path):
    '''
    创建指定的文件夹
    :param path: 文件夹路径，字符串格式
    :return: True(新建成功) or False(文件夹已存在，新建失败)
    '''
    # 引入模块
    import os

    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")

    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)

    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
         # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在')
        return False

In [4]:
def get_variables_array(objs):
    
    obj1 = objs[ak.argsort(objs.pt,axis=1,ascending=False)==0]
    obj2 = objs[ak.argsort(objs.pt,axis=1,ascending=False)==1]
    
    results = []
    results.append(obj1.pt)
    results.append(obj2.pt)
    results.append(obj1.eta)
    results.append(obj2.eta)
    results.append(obj1.phi)
    results.append(obj2.phi)
    
    results.append(Delta_eta(obj1.eta, obj2.eta))
    results.append(Delta_phi(obj1.phi, obj2.phi))
    results.append(Delta_r(obj1.eta,obj1.phi,obj2.eta,obj2.phi))
    results.append((obj1+obj2).mass)
    results.append((obj1+obj2).pt)
    
    return results

def get_zleps(leps, jets):
    
    jet1 = jets[ak.argsort(jets.pt,axis=1,ascending=False)==0]
    jet2 = jets[ak.argsort(jets.pt,axis=1,ascending=False)==1]
    
    lep1 = leps[ak.argsort(leps.pt,axis=1,ascending=False)==0]
    lep2 = leps[ak.argsort(leps.pt,axis=1,ascending=False)==1]
    
    zlep1 = abs(lep1.eta-(jet1.eta+jet2.eta)[:,0]/2)/abs(jet1.eta-jet2.eta)[:,0]
    zlep2 = abs(lep2.eta-(jet1.eta+jet2.eta)[:,0]/2)/abs(jet1.eta-jet2.eta)[:,0]
    
    return zlep1, zlep2

def get_multiobj(MET, leps, jets):
    
    variables_for_multiobj = ['dphillmet', 'dphijjmet', 'detajjll', 'dphijjll', 'drjjll']
    
    jet1 = jets[ak.argsort(jets.pt,axis=1,ascending=False)==0]
    jet2 = jets[ak.argsort(jets.pt,axis=1,ascending=False)==1]
    
    lep1 = leps[ak.argsort(leps.pt,axis=1,ascending=False)==0]
    lep2 = leps[ak.argsort(leps.pt,axis=1,ascending=False)==1]
    
    phi_jj = (jet1+jet2).phi[:,0]
    eta_jj = (jet1+jet2).eta[:,0]
    
    phi_ll = (lep1+lep2).phi[:,0]
    eta_ll = (lep1+lep2).eta[:,0]
    
    results = []
    
    results.append(Delta_phi(phi_ll, MET.phi))
    results.append(Delta_phi(phi_jj, MET.phi))
    results.append(Delta_eta(eta_jj, eta_ll))
    results.append(Delta_phi(phi_jj, phi_ll))
    results.append(Delta_r(eta_jj, phi_jj, eta_ll, phi_ll))
    
    return results

In [ ]:
#input variables: 
#  for leptons: Lepton_pt1, Lepton_pt2, Lepton_eta1, Lepton_eta2, Lepton_phi1, Lepton_phi2, detall, dphill, drll, mll, ptll
#  for jets: Jet_pt1, Jet_pt2, Jet_eta1, Jet_eta2, Jet_phi1, Jet_phi2, detajj, dphijj, drjj, mjj, ptjj
#  for MET: MET_pt, MET_phi, mt
#  for zeppenfeld: zlep1, zlep2
#  others: njets, dphillmet, dphijjmet, detajjll, dphijjll, drjjll, lepton_channel

# '1' and '2' means leading and subleading

In [5]:
ak.behavior.update(vector.behavior)
variables_for_leps = ['Lepton_pt1', 'Lepton_pt2', 'Lepton_eta1', 'Lepton_eta2', 'Lepton_phi1', 'Lepton_phi2', 'detall', 'dphill', 'drll', 'mll', 'ptll']
variables_for_jets = ['Jet_pt1', 'Jet_pt2', 'Jet_eta1', 'Jet_eta2', 'Jet_phi1', 'Jet_phi2', 'detajj', 'dphijj', 'drjj', 'mjj', 'ptjj']
variables_for_MET = ['MET_pt', 'MET_phi']
variables_for_zlep = ['zlep1', 'zlep2']
variables_for_multiobj = ['dphillmet', 'dphijjmet', 'detajjll', 'dphijjll', 'drjjll']
variables_for_others = ['njets', 'lepton_channel']

out_dir = 'array_for_training_nonprompt'
mkdir(out_dir)

# for isdata in ['data','mc']:
# for isdata in ['mc']:
for isdata in ['data']:
    nevents = 0
# for isdata in ['mc']:
# for isdata in ['data']:
#     isdata = 'data'#'data' or 'mc'
    if isdata=='data':
        Data_merge = [
                    'DoubleMuon_Run2018A','DoubleMuon_Run2018B','DoubleMuon_Run2018C','DoubleMuon_Run2018D',
                     'EGamma_Run2018A','EGamma_Run2018B','EGamma_Run2018C','EGamma_Run2018D',
                     'SingleMuon_Run2018A','SingleMuon_Run2018B','SingleMuon_Run2018C','SingleMuon_Run2018D',
                     'MuonEG_Run2018A','MuonEG_Run2018B','MuonEG_Run2018C','MuonEG_Run2018D'
                     ]
        PROCESSES= [Data_merge]

        with open(f"/data/pubfs/tyyang99/jupyter_files/data_2018_nanov7.yaml", 'r') as f:
            mc_yaml = yaml.load(f, Loader=yaml.FullLoader)
    elif isdata=='mc':
        WW_polar = ['WpWpJJ_LL_polarization_EWK','WpWpJJ_TL_polarization_EWK','WpWpJJ_TT_polarization_EWK']
        WW_merge = ['WpWpJJ_EWK', 'WpWpJJ_QCD']
        WZ_EWK = ['WLLJJ_EWK']
        WZ_merge = ['WZTo3LNu_0Jets_MLL_4to50','WZTo3LNu_1Jets_MLL_4to50','WZTo3LNu_2Jets_MLL_4to50','WZTo3LNu_3Jets_MLL_4to50',
                   'WZTo3LNu_0Jets_MLL_50','WZTo3LNu_1Jets_MLL_50','WZTo3LNu_2Jets_MLL_50','WZTo3LNu_3Jets_MLL_50']
        ZZ_merge = ['ZZJJTo4L_EWK','ZZJJTo4L_QCD','ZZJJTo4L_int','ggZZ_2e2mu',
                   'ggZZ_2e2tau','ggZZ_2mu2tau','ggZZ_4e','ggZZ_4mu','ggZZ_4tau','ggZZ_4tau_ext']
        TVX_merge = ['TTGJets','TTZToQQ','TTZToQQ_ext','TTZToLLNuNu_M_10',
                     'TTWJetsToQQ','TTWJetsToLNu','tZq']
        VG_merge = ['Zgamma_EWK','Zgamma_EWK_ext','Wgamma_EWK','Wgamma_int']
        WS_merge = ['WWTo2L2Nu',
                    'ggWW_ee','ggWW_em','ggWW_et','ggWW_me','ggWW_mm','ggWW_mt','ggWW_te','ggWW_tm','ggWW_tt',
                    'TTTo2L2Nu','ST_tW_top','ST_tW_antitop',
                    'DYJetsToLL_M50','DYJets_M10to50','DYJets_M10to50_ext','ggh_ww','ggh_zz','ggh_tautau',
                    'VBF_HToZZTo4L','VBFHToWWTo2L2Nu','VBFHToTauTau',
                    'ttHToNonbb','VHToNonbb']
        Other_merge = ['WW_DS','WWW','WWZ','WZZ','ZZZ','WWG']

        PROCESSES = [WW_polar, WW_merge, WZ_EWK, WZ_merge, ZZ_merge, TVX_merge, VG_merge, WS_merge, Other_merge]
#         PROCESSES = [WW_polar]
#         PROCESSES = [WW_merge]
        with open(f"/data/pubfs/tyyang99/jupyter_files/changexs_datasets.yaml", 'r') as f:
            mc_yaml = yaml.load(f, Loader=yaml.FullLoader)
    else:
        print('!!!Wrong data type!!!')

    parquet_dir =  f'coffea_for_training_{isdata}_parquet'
    logfile_dir = parquet_dir + '/' + 'log'
#     py_file = 'process_parquet.py'
    pwd = os.popen('pwd').readlines()[0].replace('\n','')
    
    signal_judge_list = []
    
    for i,process_list in enumerate(PROCESSES):
        for process in process_list:
            if process.endswith('_ext'):
                continue
            else:
#                 print(process)
#                 if 'nonp' in process or 'os' in process:
#                     print(process)
                file_dir = pwd + '/' + parquet_dir + '/' + process
                file_list = os.listdir(file_dir)
                outfile_dir = logfile_dir + f'/{process}/{process}.out'
                out = open(outfile_dir,'r').readlines()
                
#                 n_SR_file = 0
                
#                 for i,line in enumerate(out):
# #                     if (i+1) % 4==0:
#                     if i>0 and i%4==0:
#                         if int(line.replace('\n',''))!=0:
#                             n_SR_file+=1
                eve_file_list = []
                for eve_file in file_list:
#                     if not ('nonp' in eve_file or 'os' in eve_file):
#                     if 'os' in eve_file:
                    if 'nonp' in eve_file:
                        eve_file_list.append(eve_file)
#                 print(process,len(eve_file_list),n_SR_file)
                print('\n'+process)
#                 if n_SR_file==0:
#                     print(f'no events in {process}.')
#                     continue
                    
#                 if len(eve_file_list)!=n_SR_file:
#                     print(process,len(eve_file_list),n_SR_file)
#                     continue
                
                ##################################################################
                events_list = []
                #now the calculation just begin
                for n_eve, eve_file in enumerate(eve_file_list):
                    print('\r%d/%d'%(n_eve+1,len(eve_file_list)),end='')
                    abs_eve_file = file_dir + '/' + eve_file
#                     print(eve_file)
                    temp_events = NanoEventsFactory.from_parquet(abs_eve_file, schemaclass=NanoAODSchema).events()
                    events_list.append(temp_events)
        
                events = ak.concatenate(events_list)
                leptons = ak.zip({
                    'pt':     events.Lepton['pt'],
                    'eta':    events.Lepton['eta'],
                    'phi':    events.Lepton['phi'],
                    'mass':   events.Lepton['mass'], 
#                         'charge': events['Lepton_charge'], 
                }, with_name="PtEtaPhiMLorentzVector")
                jets = ak.zip({
                    'pt':     events.Jet['pt'],
                    'eta':    events.Jet['eta'],
                    'phi':    events.Jet['phi'],
                    'mass':   events.Jet['mass'], 
                }, with_name="PtEtaPhiMLorentzVector")
                MET = events.MET
                weight = events.Event.Weight
                channel_2e = events.Channel['2e']
                channel_2mu = events.Channel['2mu']
                channel_1e1mu = events.Channel['1e1mu']
                channel = 1*(channel_2e==1) + 2*(channel_2mu==1) + 3*(channel_1e1mu==1)
                nevents += len(channel_2e)

                leptons_variables = get_variables_array(leptons)
                jets_variables = get_variables_array(jets)
                
                dict_for_variables = {}

                for na, array in enumerate(leptons_variables):
                    iv = variables_for_leps[na]
                    dict_for_variables[iv] = array[:,0]

                for na, array in enumerate(jets_variables):
                    iv = variables_for_jets[na]
                    dict_for_variables[iv] = array[:,0]

                dict_for_variables['MET_pt'] = MET.pt
                dict_for_variables['MET_phi'] = MET.phi

                zlep1, zlep2 = get_zleps(leptons, jets)

                dict_for_variables['zlep1'] = zlep1[:,0]
                dict_for_variables['zlep2'] = zlep2[:,0]
                
                dict_for_variables['njets'] = np.sum(ak.ones_like(jets.pt),axis=1)
                dict_for_variables['lepton_channel'] = channel
                
                multi_variables = get_multiobj(MET, leptons, jets)
                
                for na, array in enumerate(multi_variables):
                    iv = variables_for_multiobj[na]
                    dict_for_variables[iv] = array
                    
                dict_for_variables['weight'] = weight
                
                dict_to_ak = ak.Array(dict_for_variables)
                ak.to_parquet(dict_to_ak,f'{out_dir}/{process}.parquet')
                
                

array_for_training_nonprompt 创建成功

DoubleMuon_Run2018A
1/37

/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for Electron_jetIdx => Jet
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for Electron_photonIdx => Photon
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx2 => SubJet
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FsrPhoton_muonIdx => Muon
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/pyt

37/37
DoubleMuon_Run2018B
12/12
DoubleMuon_Run2018C
15/15
DoubleMuon_Run2018D
67/67
EGamma_Run2018A
16/16
EGamma_Run2018B
9/9
EGamma_Run2018C
7/7
EGamma_Run2018D
55/55
SingleMuon_Run2018A
22/22
SingleMuon_Run2018B
9/9
SingleMuon_Run2018C
12/12
SingleMuon_Run2018D
62/62
MuonEG_Run2018A
28/28
MuonEG_Run2018B
12/12
MuonEG_Run2018C
14/14
MuonEG_Run2018D
50/50

In [54]:
dict_for_variables

{'Lepton_pt1': <Array [39.3, 61.4, 133, ... 97, 49.5, 164] type='155934 * float32[parameters={"...'>,
 'Lepton_pt2': <Array [29.5, 57.7, 32.9, ... 23.6, 37.9, 68.6] type='155934 * float32[parameter...'>,
 'Lepton_eta1': <Array [-1.44, -0.0688, ... 0.142, 0.627] type='155934 * float32[parameters={"__...'>,
 'Lepton_eta2': <Array [0.918, 0.871, ... -0.809, -0.317] type='155934 * float32[parameters={"__...'>,
 'Lepton_phi1': <Array [-1.11, 1.61, -1.35, ... -2.1, -2.57] type='155934 * float32[parameters={...'>,
 'Lepton_phi2': <Array [2.95, -2.42, 1.46, ... -1.02, -0.131] type='155934 * float32[parameters=...'>,
 'detall': array([2.3535156 , 0.9397888 , 0.45837402, ..., 1.4099731 , 0.95077515,
        0.94421387], dtype=float32),
 'dphill': array([2.2280097 , 2.26121283, 2.80395508, ..., 2.62670898, 1.07788086,
        2.43832397]),
 'drll': array([3.24084295, 2.4487316 , 2.8411742 , ..., 2.98121189, 1.43728931,
        2.61475881]),
 'mll': <Array [117, 122, 134, 115, ... 118, 61.7, 225] 

In [55]:
weight

<Array [0.000478, 0.000482, ... 0.000479] type='155934 * float32[parameters={"__...'>